🏡 Kaggle Notebook: House Price Prediction with Lasso, XGBoost & Stacking

# 🏡 House Prices: Advanced Regression Techniques

This notebook walks through a **complete end-to-end pipeline** for predicting house prices.  
We cover:

1. Data Loading & Exploration  
2. Handling Missing Values  
3. Feature Engineering  
4. Encoding & Scaling  
5. Model Training: **LassoCV** & **XGBoost**  
6. Blending Models  
7. **Stacking with Ridge meta-model (Best Score: ~0.11845 RMSE)** 


In [14]:
# ================================================================
# 1. Imports & setup
# ================================================================
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso, Ridge
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

SEED = 42
N_SPLITS = 5


## 📂 Step 2: Load Data
We load the **train** and **test** datasets provided by Kaggle.  
Target variable: `SalePrice` (we will log-transform it for stability).


In [15]:
DATA_DIR = Path(r"C:\Users\chand\Desktop\ML Projects\Kaggle- house-price")
train = pd.read_csv(DATA_DIR / "train.csv")
test  = pd.read_csv(DATA_DIR / "test.csv")

print("Train:", train.shape, " Test:", test.shape)

# Target on log scale for stable training
y = np.log1p(train["SalePrice"])
train_id = train["Id"].copy()
test_id  = test["Id"].copy()

# Work on copies; drop identifiers/target from features
X_tr_raw = train.drop(columns=["Id", "SalePrice"]).copy()
X_te_raw = test.drop(columns=["Id"]).copy()


Train: (1460, 81)  Test: (1459, 80)


## 3) Preprocessing (safe leak-free wrt target)
We **concatenate train+test** to ensure consistent encoding & imputation across both splits (this is allowed in Kaggle since test features are known and the target is not used).

Strategies:
- Fill domain “no feature” categoricals with `"None"`
- Fill numerical **no-feature** columns with `0`
- `LotFrontage` by neighborhood median
- Remaining categoricals → mode; remaining numerics → median
- Cast `MSSubClass` to string (categorical)


In [16]:
all_data = pd.concat([X_tr_raw, X_te_raw], axis=0).reset_index(drop=True)

# treat MSSubClass as categorical
all_data["MSSubClass"] = all_data["MSSubClass"].astype(str)

# 3.1 categorical None-fill
none_cols = ["Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
             "FireplaceQu","GarageType","GarageFinish","GarageQual","GarageCond",
             "PoolQC","Fence","MiscFeature"]
for c in none_cols:
    if c in all_data.columns:
        all_data[c] = all_data[c].fillna("None")

# 3.2 numerical zero-fill (absence)
zero_cols = ["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF",
             "BsmtFullBath","BsmtHalfBath","GarageYrBlt","GarageCars","GarageArea"]
for c in zero_cols:
    if c in all_data.columns:
        all_data[c] = all_data[c].fillna(0)

# 3.3 LotFrontage by neighborhood median
if "LotFrontage" in all_data.columns and "Neighborhood" in all_data.columns:
    all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"] \
                                      .transform(lambda x: x.fillna(x.median()))

# 3.4 remaining
for c in all_data.select_dtypes(include="object").columns:
    all_data[c] = all_data[c].fillna(all_data[c].mode()[0])
for c in all_data.select_dtypes(exclude="object").columns:
    all_data[c] = all_data[c].fillna(all_data[c].median())

## 4) Feature engineering
Some additions for this dataset:
- `TotalSF` (total livable area)
- `TotalBath` (full + 0.5×half, incl. basement)
- `OverallQual_GrLivArea` (quality × size).
We also drop a few noisy/low-informative columns


In [17]:
# New features
all_data["TotalSF"] = all_data["TotalBsmtSF"] + all_data["1stFlrSF"] + all_data["2ndFlrSF"]
all_data["TotalBath"] = (all_data["FullBath"] + 0.5*all_data["HalfBath"] +
                         all_data["BsmtFullBath"] + 0.5*all_data["BsmtHalfBath"])
all_data["OverallQual_GrLivArea"] = all_data["OverallQual"] * all_data["GrLivArea"]

# Drop a few consistently unhelpful columns
drop_cols = ["PoolArea","MiscVal","3SsnPorch","LowQualFinSF"]
drop_more = ["Utilities","Street","Condition2","RoofMatl","Heating",
             "GarageArea",           # keep GarageCars
             "TotRmsAbvGrd"]         # keep TotalRooms proxy features
for c in drop_cols + drop_more:
    if c in all_data.columns:
        all_data = all_data.drop(columns=c)


## 🔢 Step 5: Encoding
We encode categorical features:
- Ordinal encoding for quality ratings.  
- One-hot encoding for all remaining categorical features.  


In [18]:
qual_map = {"Ex":5, "Gd":4, "TA":3, "Fa":2, "Po":1, "None":0}
ordinal_cols = ["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC",
                "KitchenQual","FireplaceQu","GarageQual","GarageCond","PoolQC"]
for c in ordinal_cols:
    if c in all_data.columns:
        all_data[c] = all_data[c].map(qual_map)

all_data = pd.get_dummies(all_data, drop_first=True)

print("All-data shape after encoding:", all_data.shape)


All-data shape after encoding: (2919, 218)


## 6) Fix skewness (numeric only)
Log1p-transform skewed numeric features (|skew| > 0.75).


In [19]:
numeric_cols = all_data.select_dtypes(include=[np.number]).columns
skewed = all_data[numeric_cols].apply(lambda x: x.dropna().skew()).abs()
skew_cols = skewed[skewed > 0.75].index
all_data[skew_cols] = np.log1p(all_data[skew_cols])

## 7) Split back into train/test and prepare scalers for linear models



In [20]:
n_train = X_tr_raw.shape[0]
X_train = all_data.iloc[:n_train, :].copy()
X_test  = all_data.iloc[n_train:, :].copy()

# We'll scale inside each CV fold for Lasso when creating OOF predictions
print("Train matrix:", X_train.shape, " Test matrix:", X_test.shape)


Train matrix: (1460, 218)  Test matrix: (1459, 218)


## 8) Helper: OOF predictions (essential for correct stacking)
This function returns:
- `oof_log`: OOF predictions on **log scale** aligned to `y`
- `test_log_mean`: mean test prediction across folds on **log scale**


In [21]:
def get_oof_lasso(X, y, X_test, seed=SEED, n_splits=N_SPLITS):
    # tune alpha once on a global (train-only) scale version to keep it fast/reliable
    scaler_global = StandardScaler().fit(X)
    X_scaled_global = scaler_global.transform(X)

    lasso_cv = LassoCV(alphas=np.logspace(-4, -0.5, 50),
                       cv=5, max_iter=50000, random_state=seed)
    lasso_cv.fit(X_scaled_global, y)
    best_alpha = lasso_cv.alpha_
    print(f"[Lasso] Best alpha: {best_alpha:.6f}")

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X))
    test_fold_pred = np.zeros((n_splits, X_test.shape[0]))

    for i, (tr_idx, va_idx) in enumerate(kf.split(X)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr = y.iloc[tr_idx]

        # scale per-fold to avoid leakage
        scaler = StandardScaler().fit(X_tr)
        X_tr_sc = scaler.transform(X_tr)
        X_va_sc = scaler.transform(X_va)
        X_te_sc = scaler.transform(X_test)

        model = Lasso(alpha=best_alpha, max_iter=50000, random_state=seed)
        model.fit(X_tr_sc, y_tr)

        oof[va_idx] = model.predict(X_va_sc)             # already in log scale
        test_fold_pred[i, :] = model.predict(X_te_sc)    # log scale

    oof_log = oof
    test_log_mean = test_fold_pred.mean(axis=0)
    rmse = np.sqrt(mean_squared_error(y, oof_log))
    print(f"[Lasso] OOF RMSE (log): {rmse:.5f}")
    return oof_log, test_log_mean, best_alpha


def get_oof_xgb(X, y, X_test, seed=SEED, n_splits=N_SPLITS):
    params = dict(
        n_estimators=2000,
        learning_rate=0.01,
        max_depth=4,
        subsample=0.7,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=seed,
        tree_method="hist"
    )
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    oof = np.zeros(len(X))
    test_fold_pred = np.zeros((n_splits, X_test.shape[0]))

    for i, (tr_idx, va_idx) in enumerate(kf.split(X)):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr = y.iloc[tr_idx]
        model = XGBRegressor(**params)
        model.fit(X_tr, y_tr, verbose=False)

        oof[va_idx] = model.predict(X_va)          # log scale
        test_fold_pred[i, :] = model.predict(X_test)

    oof_log = oof
    test_log_mean = test_fold_pred.mean(axis=0)
    rmse = np.sqrt(mean_squared_error(y, oof_log))
    print(f"[XGB ] OOF RMSE (log): {rmse:.5f}")
    return oof_log, test_log_mean, params


## 9) Train base models & create submissions
- `submission_blend.csv` — a simple 0.35/0.65 blend (good baseline)
- `submission_stack.csv` — Ridge stacker on OOFs (your best previously)


In [26]:
# Base OOFs (LOG scale)
lasso_oof_log, lasso_test_log, best_alpha = get_oof_lasso(X_train, y, X_test)
xgb_oof_log,   xgb_test_log,   xgb_params   = get_oof_xgb(X_train, y, X_test)

# Blend on ORIGINAL scale for a baseline submission
lasso_test_pred = np.expm1(lasso_test_log)
xgb_test_pred   = np.expm1(xgb_test_log)

BLEND_W = 0.35  # (found good in prior experiments)
blend_pred = BLEND_W * lasso_test_pred + (1 - BLEND_W) * xgb_test_pred

pd.DataFrame({"Id": test_id, "SalePrice": blend_pred}).to_csv("submission_blend2.csv", index=False)
print("Saved: submission_blend2.csv")

[Lasso] Best alpha: 0.003728
[Lasso] OOF RMSE (log): 0.13161
[XGB ] OOF RMSE (log): 0.12805
Saved: submission_blend2.csv


In [27]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission_blend2.csv -m "LassoCV xgboostCV blend submission"

Successfully submitted to House Prices - Advanced Regression Techniques



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
 46%|████▌     | 16.0k/35.1k [00:00<00:00, 90.9kB/s]
100%|██████████| 35.1k/35.1k [00:01<00:00, 30.7kB/s]


## 10) Stacking (Ridge meta-model)
- Train meta on **OOF (log)** predictions  
- Predict test on **mean fold test preds (log)**  
- Convert final to **original scale** for submission


In [23]:
# Stack training matrix (log scale features)
stack_train = np.vstack([lasso_oof_log, xgb_oof_log]).T
stack_test  = np.vstack([lasso_test_log, xgb_test_log]).T

meta = Ridge(alpha=1.0, random_state=SEED)
meta.fit(stack_train, y)

# CV estimate on stacker (optional quick sanity check)
stack_pred_oof = meta.predict(stack_train)
stack_rmse_log = np.sqrt(mean_squared_error(y, stack_pred_oof))
print(f"[Stack] OOF RMSE (log): {stack_rmse_log:.5f}")

# Test predictions
stack_pred_log = meta.predict(stack_test)
stack_pred = np.expm1(stack_pred_log)

pd.DataFrame({"Id": test_id, "SalePrice": stack_pred}).to_csv("submission_stack3.csv", index=False)
print("Saved: submission_stack3.csv")

[Stack] OOF RMSE (log): 0.12534
Saved: submission_stack3.csv


# ✅ Conclusion
- We built a **robust pipeline** for house price prediction.  
- Preprocessing: handled missing values, feature engineering, encoding, scaling.  
- Models: LassoCV, XGBoost, Blending, and **Stacking**.  
- Best performance came from **Stacking with Ridge meta-model** (~0.1184 RMSE).  

🚀 Next steps:  
- Try **LightGBM** / **CatBoost**.  
- More **feature interactions**.  
- Hyperparameter tuning with **Optuna**.  